In [1]:
import os
os.chdir('..')

In [ ]:
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor, as_completed
from datetime import timedelta

import polars as pl

import config
from aws.kline.util import local_list_kline_symbols
from config import BINANCE_DATA_DIR, TradeType
from merge.kline import merge_klines

pl.Config.set_tbl_rows(30)

In [3]:
symbol = 'BNXUSDT'
time_interval = '1m'

In [ ]:
def scan_gaps(df: pl.DataFrame, min_delta: timedelta, min_price_chg: float) -> pl.DataFrame:
    ldf = df.lazy()

    ldf = ldf.with_columns(
        pl.col("candle_begin_time").diff().alias("time_diff"),
        (pl.col("open") / pl.col("close").shift() - 1).alias("price_change"),
        pl.col("candle_begin_time").shift().alias("prev_begin_time"),
        pl.col('close').shift().alias('prev_close')
    )

    df_gap = ldf.filter((pl.col("time_diff") > min_delta) & (pl.col("price_change").abs() > min_price_chg))
    df_gap = df_gap.select("prev_begin_time", "candle_begin_time", 'prev_close', "open", "time_diff", "price_change")

    return df_gap.collect()


min_delta = timedelta(days=1)
min_price_chg = 0.1

df = merge_klines(TradeType.spot, "BNXUSDT", time_interval, exclude_empty=True)
display(scan_gaps(df, min_delta, min_price_chg))

df = merge_klines(TradeType.spot, "BTCUSDT", time_interval, exclude_empty=True)
display(scan_gaps(df, min_delta, min_price_chg))

In [ ]:
def display_gaps_all(trade_type: TradeType, time_interval: str, min_delta: timedelta, min_price_chg: float):
    symbols = local_list_kline_symbols(trade_type, time_interval)

    dfs = []
    for symbol in symbols:
        ldf = merge_klines(TradeType.spot, symbol, time_interval, exclude_empty=True)
        df_gap = scan_gaps(ldf, min_delta, min_price_chg)
        if not df_gap.is_empty():
            dfs.append(df_gap.with_columns(pl.lit(symbol).alias("symbol")))

    df_gaps =pl.concat(dfs).sort('candle_begin_time')
    display(df_gaps)

display_gaps_all(TradeType.spot, '1m', timedelta(days=1), 0.1)
display_gaps_all(TradeType.spot, '1m', timedelta(days=2), 0)
display_gaps_all(TradeType.spot, '1m', timedelta(days=1), 0)


In [ ]:
min_delta = timedelta(days=3)
min_price_chg = 0
trade_type = TradeType.spot

symbols = local_list_kline_symbols(trade_type, time_interval)

dfs = []
for symbol in symbols:
    ldf = merge_klines(TradeType.spot, symbol, time_interval, exclude_empty=True)
    df_gap = scan_gaps(ldf, min_delta, min_price_chg)
    if not df_gap.is_empty():
        print(symbol)
        dfs.append(df_gap.with_columns(pl.lit(symbol).alias("symbol")))

pl.concat(dfs).sort('candle_begin_time')

In [ ]:
min_delta = timedelta(days=3)
min_price_chg = 0
trade_type = TradeType.spot

symbols = (BINANCE_DATA_DIR / "parsed_data" / trade_type.value / "klines").glob("*")
symbols = sorted(s.name for s in symbols)

dfs = []
for symbol in symbols:
    ldf = merge_klines(TradeType.spot, symbol, time_interval, exclude_empty=True)
    df_gap = scan_gaps(ldf, min_delta, min_price_chg)
    if not df_gap.is_empty():
        print(symbol)
        dfs.append(df_gap.with_columns(pl.lit(symbol).alias("symbol")))

pl.concat(dfs).sort('candle_begin_time')

In [ ]:
min_delta = timedelta(days=1)
min_price_chg = 0.1
trade_type = TradeType.um_futures

symbols = (BINANCE_DATA_DIR / "parsed_data" / trade_type.value / "klines").glob("*")
symbols = sorted(s.name for s in symbols)

dfs = []
for symbol in symbols:
    df_gap = scan_gaps(trade_type, symbol, time_interval, min_delta, min_price_chg)
    if not df_gap.is_empty():
        print(symbol)
        dfs.append(df_gap.with_columns(pl.lit(symbol).alias("symbol")))

pl.concat(dfs)

In [ ]:
min_delta = timedelta(days=3)
min_price_chg = 0
trade_type = TradeType.um_futures

symbols = (BINANCE_DATA_DIR / "parsed_data" / trade_type.value / "klines").glob("*")
symbols = sorted(s.name for s in symbols)

dfs = []
for symbol in symbols:
    df_gap = scan_gaps(trade_type, symbol, time_interval, min_delta, min_price_chg)
    if not df_gap.is_empty():
        print(symbol)
        dfs.append(df_gap.with_columns(pl.lit(symbol).alias("symbol")))

pl.concat(dfs)